# Reproduce main figures

In [1]:
!python3 nb.py --bases experiment_archive results --out reports/week01

Scanning experiment_archive ...


Scanning results ...


/workspace/nb.py:1092: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, showfliers=False)


/workspace/nb.py:1100: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, showfliers=False)


Done. See: reports/week01/summary.md
